In [14]:
import openai
import pymupdf4llm
import pymupdf
import json
from fuzzysearch import find_near_matches
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd
import openai
import json
import pymupdf4llm
import pymupdf

from comments import (
    find_comments_in_doc,
    get_historic_sites_comments,
    get_species_comments,
    get_litigation_comments,
)

from tqdm import tqdm

In [2]:
PAGES_PER_CHUNK = 50

In [3]:
# Load EIS document
page_count = pymupdf.open("sample.pdf").page_count
full_doc = pymupdf4llm.to_markdown("sample.pdf")
doc_chunks = [
    pymupdf4llm.to_markdown("sample.pdf", pages=range(i, i + PAGES_PER_CHUNK))
    for i in range(0, page_count, PAGES_PER_CHUNK)
]

In [4]:
# Load context
sites = pd.read_excel("sources/sites.xlsx")
species = pd.read_csv("sources/species.csv")
litigations = json.load(open("sources/litigation.json"))

In [19]:
final_comments = []
for chunk in tqdm(doc_chunks):
    site_comments = get_historic_sites_comments(chunk, sites)
    species_comments = get_species_comments(chunk, species)
    litigation_comments = get_litigation_comments(chunk, litigations)
    chunk_comments = (
        litigation_comments["comments"]
        + site_comments["comments"]
        + species_comments["comments"]
    )
    final_comments.extend(chunk_comments)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [03:34<00:00, 35.82s/it]


In [20]:
found_comments = find_comments_in_doc(full_doc, final_comments)
found_comments

[{'quote': {'start': 23557,
   'end': 23735,
   'text': 'In a separate process, Indian Tribes have also\nbeen invited to participate specifically in the NEPA review and under Section 106 of the National Historic\nPreservation Act (NHPA).'},
  'comment': 'The document mentions compliance with Section 106 of the National Historic Preservation Act. This indicates that the project team is aware of and attempting to meet regulatory requirements for historic and cultural resource preservation. The developer should ensure full compliance with Section 106, including thorough consultations, documentation, and mitigation strategies for any adverse effects identified.'},
 {'quote': {'start': 23737,
   'end': 24036,
   'text': 'The BLM is conducting on-going government-to-government consultation with Bridgeport Indian Colony,\nFallon Paiute-Shoshone Tribe, Pyramid Lake Paiute Tribe, Reno-Sparks Indian Colony, Walker River Paiute\nTribe, Washoe Tribe of Nevada and California, Yerington Paiute Tribe

In [21]:
len(final_comments), len(found_comments)

(60, 45)

In [ ]:
output = {"markdown": full_doc, "comments": found_comments}
with open("final_2.json", "w") as f:
    f.write(json.dumps(output, indent=4))